# TASK 1 : Install Dependencies & Run a SparkSession


In [ ]:
#install pyspark
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=eff2d1c224b3dc813bd46c3aec5035ce9d09b26129d1bdeedab2515a1827fcce
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
#create a sparksession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("spark").getOrCreate()

# TASK 2 : Clone & Explore dataset

In [ ]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 5.60 KiB | 5.60 MiB/s, done.


In [ ]:
#check the presence of dataset
! ls admission_dataset

Admission_Predict_Ver1.1.csv


In [9]:
#create a spark dataframe
df=spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv', header=True,inferSchema=True)

In [10]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [11]:
#get the no.of rows & columns
print((df.count(),len(df.columns)))

(500, 9)


In [12]:
#print schema
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [13]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

# TASK 3 : Data Cleaning

In [14]:
#drop the unnecessary column
df= df.drop('Serial No')

In [15]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [16]:
#check for null values
for i in df.columns:
  print(i+":",df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


# TASK 4 : Correlation Analysis & Feature Selection

In [17]:
# correlation analysis
for col in df.columns:
  print('Correlation to chance to admit col for {} is {}'.format(col,df.stat.corr('Chance of Admit',col)))

Correlation to chance to admit col for GRE Score is 0.8103506354632598
Correlation to chance to admit col for TOEFL Score is 0.7922276143050823
Correlation to chance to admit col for University Rating is 0.6901323687886892
Correlation to chance to admit col for SOP is 0.6841365241316723
Correlation to chance to admit col for LOR is 0.6453645135280112
Correlation to chance to admit col for CGPA is 0.882412574904574
Correlation to chance to admit col for Research is 0.5458710294711379
Correlation to chance to admit col for Chance of Admit is 1.0


In [21]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler= VectorAssembler(inputCols=['GRE Score','TOEFL Score','CGPA'],outputCol='features')

In [22]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# TASK 5 : Build the Linear Regression Model

In [23]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data=output_data.select('features','Chance of Admit')

In [24]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [26]:
#split the dataset into training and testing set
train, test =final_data.randomSplit([0.7,0.3])

In [27]:
#build & train the model
models=LinearRegression(featuresCol='features',labelCol='Chance of Admit')
model=models.fit(train)

In [28]:
#get coefficients & intercept
print("Coefficients:",model.coefficients)
print('intercept:',model.intercept)

Coefficients: [0.0022923443888241118,0.0029384264776114058,0.1510396383711094]
intercept: -1.6153831690876639


In [29]:
#get summary of the model
summary=model.summary

In [30]:
#print the rmse & r2 score
print('RMSE',summary.rootMeanSquaredError)
print('r2 score',summary.r2)

RMSE 0.06290862935932613
r2 score 0.80868586902768


# TASK 6 : Evaluate & Save the Model

In [31]:
#transform on the test data
predictions=model.transform(test)

In [32]:
#display the predictions
predictions.show(20)

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|[290.0,104.0,7.46]|           0.45|0.48174875959139074|
| [294.0,95.0,7.64]|           0.49| 0.4916594337549842|
|  [295.0,93.0,7.2]|           0.46|0.42161748430529755|
|[296.0,101.0,7.68]|            0.6| 0.5199162669331452|
| [297.0,96.0,7.89]|           0.43| 0.5392348029918455|
| [297.0,98.0,7.67]|           0.59| 0.5118829355054242|
| [297.0,99.0,7.81]|           0.54| 0.5359669113549907|
| [298.0,92.0,7.88]|           0.51| 0.5282630450865127|
| [298.0,99.0,7.46]|           0.53|0.48539538231392676|
|[298.0,100.0,7.95]|           0.58| 0.5623432315933816|
|[298.0,101.0,7.86]|           0.54| 0.5516880906175934|
|[298.0,105.0,8.54]|           0.69| 0.6661487506203931|
|[299.0,100.0,7.88]|           0.68| 0.5540628012962281|
|[299.0,102.0,8.62]|           0.56| 0.6717089866460717|
|  [300.0,97.0,8.1]|           

In [33]:
#evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator
evaluator= RegressionEvaluator(predictionCol='prediction', labelCol='Chance of Admit',metricName='r2')
print('r2 on the test data', evaluator.evaluate(predictions))

r2 on the test data 0.7927431147279084


In [34]:
#save the model
model.save("model")

In [36]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
model=LinearRegressionModel.load('model')